# Extract Audio Feautures


In [ ]:
import pickle
import pathlib
import random

import wave
import python_speech_features as ps

import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf

from util import *


In [2]:
import wave
import python_speech_features as ps

In [3]:
print("Version: ", tf.__version__)
# tf.compat.v1.enable_eager_execution() # Enable eager execution
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.experimental.list_physical_devices('GPU') else "NOT AVAILABLE")
# print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

Version:  1.14.0
Eager mode:  False
GPU is NOT AVAILABLE


## File paths

In [5]:
HOME_DIR = pathlib.Path.cwd()

# feature_path = HOME_DIR / 'data' / 'processed'/ 'IEMOCAP' / 'extracted_feature.pk'
feature_path = 'D:/extracted_features.pk'

label_path = HOME_DIR / 'data' / 'processed' / 'IEMOCAP' / 'FC_label_test.txt'
processed_id_path = HOME_DIR / 'data' / 'processed' / 'IEMOCAP' / 'processed_ids_test.txt'
dataset_path = HOME_DIR / 'data' / 'raw'


In [6]:
def read_file(filename):
    file = wave.open(filename, 'r')
    params = file.getparams()
    nchannels, sampwidth, framerate, wav_length = params[:4]
    str_data = file.readframes(wav_length)
    wavedata = np.fromstring(str_data, dtype=np.short)
    time = np.arange(0, wav_length) * (1.0/framerate)
    file.close()
    return wavedata, time, framerate


In [7]:
def shuffle_dataset(features, labels):
    # random.seed(24)

    shuffler = np.random.permutation(len(features))
    return features[shuffler], labels[shuffler]


In [8]:
list_files = []

for x in range(1, 5):
    sess_title = 'Session' + str(x)

    path = f'./data/raw/IEMOCAP_full_release/{sess_title}/sentences/wav'

    file_search(str(path), list_files)
    list_files = sorted(list_files)

    print(f"{sess_title}, # Num of files: {len(list_files)}")


Session1, # Num of files: 1819
Session2, # Num of files: 3630
Session3, # Num of files: 5766
Session4, # Num of files: 7869


## Get list of filenames in category, (Angry, Happy, Sad, Neutral)

In [18]:
full_ids = []
with open(str(processed_id_path)) as f:
    full_ids = f.readlines()
full_ids = [x.strip() for x in full_ids]

In [19]:
full_path = []

for file_ in full_ids:
    for file2 in list_files:
        if file_+'.wav' == file2.split('\\')[-1]:
            full_path.append(file2)
            break

## Extract and save features

In [22]:
def load_data():
    with open('./zscore40.pkl', 'rb') as f:
        mean1, std1, mean2, std2, mean3, std3 = pickle.load(f)
    return mean1, std1, mean2, std2, mean3, std3


In [25]:
mean1, std1, mean2, std2, mean3, std3 = load_data()

In [26]:
# dataset_size = 3548 #3548 - consider zero-indexing
dataset_size = 942  # consider session 5 only(test set)
filter_num = 40
eps = 1e-5

dataset = np.zeros((dataset_size, 300, filter_num, 3), dtype=np.float32)


In [27]:
dataset_size = 0
for filename in full_path:

    data, time, rate = read_file(filename)
    mel_spec = ps.logfbank(data, rate, nfilt=40)
    delta1 = ps.delta(mel_spec, 2)
    delta2 = ps.delta(delta1, 2)

    time = mel_spec.shape[0]

    if(time <= 300):
        part = mel_spec
        delta11 = delta1
        delta21 = delta2

        part = np.pad(
            part, ((0, 300 - part.shape[0]), (0, 0)), 'constant', constant_values=0)
        delta11 = np.pad(
            delta11, ((0, 300 - delta11.shape[0]), (0, 0)), 'constant', constant_values=0)
        delta21 = np.pad(
            delta21, ((0, 300 - delta21.shape[0]), (0, 0)), 'constant', constant_values=0)

        dataset[dataset_size, :, :, 0] = (part - mean1)/(std1+eps)
        dataset[dataset_size, :, :, 1] = (delta11 - mean2)/(std2+eps)
        dataset[dataset_size, :, :, 2] = (delta21 - mean3)/(std3+eps)

        dataset_size += 1

    else:
        for i in range(1):
            if(i == 0):
                begin = 0
                end = begin + 300
            else:
                begin = time - 300
                end = time

            part = mel_spec[begin:end, :]
            delta11 = delta1[begin:end, :]
            delta21 = delta2[begin:end, :]

            dataset[dataset_size, :, :, 0] = (part - mean1)/(std1+eps)
            dataset[dataset_size, :, :, 1] = (delta11 - mean2)/(std2+eps)
            dataset[dataset_size, :, :, 2] = (delta21 - mean3)/(std3+eps)

            dataset_size += 1


C:\Users\Lenovo\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  


## Load and prepare features

In [30]:
# Load label data
label = []

with open(str(label_path)) as f2:
    category = f2.readlines()

label = [y.strip() for y in category]


In [32]:
# Create label Dataframe
data_dict = {'label': label}
label_dataset = pd.DataFrame.from_dict(data_dict)


In [33]:
encoder = OneHotEncoder(sparse=False)
label_encoded = encoder.fit_transform(label_dataset[['label']])


## Shuffle features and label

In [22]:
features, label = shuffle_dataset(dataset, label_encoded)


In [27]:
# Save pre-processed data
np.save('D:/processed_features3', features)
np.save('D:/label3', label)


In [34]:
# Hold out test-set
np.save(str(dataset_path) + '/X_test_set55', dataset)
np.save(str(dataset_path) + '/Y_test_set55', label_encoded)
